In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO

In [2]:
df = pd.read_csv('./product_info.csv')

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,id,uid,name,keyword,category_id
0,100002112,5016511,[66] 끌리오 자켓,"66,끌리오,자켓",310100999.0
1,100013888,7680369,🔥당일배송가능🔥(260~280)구찌 모던 로퍼,"구지,구지신발,구찌신발,구찌슬리퍼,구지로퍼",400051300.0
2,100014144,4429338,⭐️구* 클러치⭐️,"구찌,구찌클러치,구찌클러치백,구찌가방,구찌핸드백",400020400.0
3,100014912,3550416,the izzat 봄 트위드자켓,"트위드자켓,트위드코트,봄코트,봄자켓,봄아우터",310100030.0
4,100021056,5811221,▷에르메스 오즈 켈리 뮬 로퍼 6color,"에르메스오즈,에르메스오즈뮬,에르메스켈리,에르메스로퍼,에르메스뮬",400040300.0


In [4]:
import re

def preprocess(name):
    name = str(name)
    regrex_pattern = re.compile(pattern = "["
    u"\U0001F600-\U0001F64F"  
    u"\U0001F300-\U0001F5FF"  
    u"\U0001F680-\U0001F6FF"  
    u"\U0001F1E0-\U0001F1FF"  
                       "]+", flags = re.UNICODE)
    name = regrex_pattern.sub(r'',name)
    name = re.sub(r'[^\w\s]', ' ', name)
    name = re.sub(r'\s+', ' ', name)
    return name.lower().strip()

In [5]:
df['preprocessed'] = df['name'].map(preprocess)

In [7]:
df.head()

,id,uid,name,keyword,category_id,preprocessed
0,100002112,5016511,[66] 끌리오 자켓,"66,끌리오,자켓",310100999.0,66 끌리오 자켓
1,100013888,7680369,🔥당일배송가능🔥(260~280)구찌 모던 로퍼,"구지,구지신발,구찌신발,구찌슬리퍼,구지로퍼",400051300.0,당일배송가능 260 280 구찌 모던 로퍼
2,100014144,4429338,⭐️구* 클러치⭐️,"구찌,구찌클러치,구찌클러치백,구찌가방,구찌핸드백",400020400.0,구 클러치
3,100014912,3550416,the izzat 봄 트위드자켓,"트위드자켓,트위드코트,봄코트,봄자켓,봄아우터",310100030.0,the izzat 봄 트위드자켓
4,100021056,5811221,▷에르메스 오즈 켈리 뮬 로퍼 6color,"에르메스오즈,에르메스오즈뮬,에르메스켈리,에르메스로퍼,에르메스뮬",400040300.0,에르메스 오즈 켈리 뮬 로퍼 6color


In [6]:
import sentencepiece as sp

In [9]:
bpe_20000 = sp.SentencePieceProcessor()
vocab_file = "name_bpe_20000.model"
bpe_20000.load(vocab_file)

True

In [11]:
df['bpe_20000_pieces'] = df['preprocessed'].apply(lambda x: bpe_20000.encode_as_pieces(x))

In [13]:
df['bpe_20000_ids'] = df['preprocessed'].apply(lambda x: bpe_20000.encode_as_ids(x))

In [14]:
unigram_20000 = sp.SentencePieceProcessor()
vocab_file = "name_unigram_20000.model"
unigram_20000.load(vocab_file)

True

In [15]:
df['unigram_20000_pieces'] = df['preprocessed'].apply(lambda x: unigram_20000.encode_as_pieces(x))
df['unigram_20000_ids'] = df['preprocessed'].apply(lambda x: unigram_20000.encode_as_ids(x))

In [16]:
df.head()

,id,uid,name,keyword,category_id,preprocessed,bpe_20000_pieces,bpe_20000_ids,unigram_20000_pieces,unigram_20000_ids
0,100002112,5016511,[66] 끌리오 자켓,"66,끌리오,자켓",310100999.0,66 끌리오 자켓,"[▁66, ▁끌, 리오, ▁자켓]","[182, 1930, 1932, 52]","[▁66, ▁끌, 리오, ▁자켓]","[40, 13529, 4951, 11]"
1,100013888,7680369,🔥당일배송가능🔥(260~280)구찌 모던 로퍼,"구지,구지신발,구찌신발,구찌슬리퍼,구지로퍼",400051300.0,당일배송가능 260 280 구찌 모던 로퍼,"[▁당일배송, 가능, ▁260, ▁280, ▁구찌, ▁모던, ▁로퍼]","[889, 508, 552, 765, 104, 2146, 673]","[▁당일배송, 가능, ▁260, ▁280, ▁구찌, ▁모던, ▁로퍼]","[383, 503, 188, 317, 23, 1343, 261]"
2,100014144,4429338,⭐️구* 클러치⭐️,"구찌,구찌클러치,구찌클러치백,구찌가방,구찌핸드백",400020400.0,구 클러치,"[▁구, ▁클러치]","[36, 261]","[▁구, ▁클러치]","[253, 132]"
3,100014912,3550416,the izzat 봄 트위드자켓,"트위드자켓,트위드코트,봄코트,봄자켓,봄아우터",310100030.0,the izzat 봄 트위드자켓,"[▁the, ▁izzat, ▁봄, ▁트위드자켓]","[2316, 12481, 282, 2113]","[▁the, ▁izzat, ▁봄, ▁트위드자켓]","[1594, 10221, 192, 1150]"
4,100021056,5811221,▷에르메스 오즈 켈리 뮬 로퍼 6color,"에르메스오즈,에르메스오즈뮬,에르메스켈리,에르메스로퍼,에르메스뮬",400040300.0,에르메스 오즈 켈리 뮬 로퍼 6color,"[▁에르메스, ▁오즈, ▁켈리, ▁뮬, ▁로퍼, ▁6, color]","[710, 2006, 7061, 1342, 673, 109, 309]","[▁에르메스, ▁오즈, ▁켈리, ▁뮬, ▁로퍼, ▁6, color]","[277, 9792, 4856, 804, 261, 227, 87]"


In [17]:
df.to_csv('df_compare_sp.csv', index=False)

In [18]:
df.sample(n=1000).to_csv('df_compare_sp_sample.csv', index=False)

In [19]:
sample = pd.read_csv('df_compare_sp_sample.csv')

,id,uid,name,keyword,category_id,preprocessed,bpe_20000_pieces,bpe_20000_ids,unigram_20000_pieces,unigram_20000_ids
0,145749070,7371587.0,바인드 니트,"bind,니트,여성니트,아이보리니트,바인드",310060040.0,바인드 니트,"['▁바인', '드', '▁니트']","[9751, 18897, 26]","['▁바인드', '▁니트']","[18668, 7]"
1,123081835,4253630.0,빨질레리 사르토리알,"란스미어키톤,라르디니링자켓,볼리올리폴로",320090100.0,빨질레리 사르토리알,"['▁빨질레리', '▁사', '르', '토리', '알']","[14404, 215, 19007, 1534, 19123]","['▁빨질레리', '▁사', '르', '토리', '알']","[12041, 1307, 528, 5801, 2220]"
2,91019252,5342587.0,긴팔 티셔츠 받쳐입기 좋아요,"66,긴팔,티셔츠",310010020.0,긴팔 티셔츠 받쳐입기 좋아요,"['▁긴팔', '▁티셔츠', '▁받', '쳐', '입기', '▁좋아요']","[395, 117, 8316, 19396, 9585, 5268]","['▁긴팔', '▁티셔츠', '▁', '받', '쳐', '입기', '▁좋아요']","[216, 25, 5, 6880, 3292, 14325, 3636]"
3,126327684,192530.0,[정품] FENDI 기저귀가방 추천,"펜디정품,펜디,펜디기저귀가방,기저귀가방추천,명품가방",400010100.0,정품 fendi 기저귀가방 추천,"['▁정품', '▁fendi', '▁기저귀가방', '▁추천']","[33, 5748, 10998, 1827]","['▁정품', '▁fendi', '▁기저귀가방', '▁추천']","[6, 4025, 8804, 1363]"
4,147514337,6612036.0,"나이키 베이퍼맥스 파슬실버 /낫띵리튼,텔파,아모멘토,시엔느,단톤","빔즈보이,네이더스,메종마르엘라,단톤,블랭코브",400040300.0,나이키 베이퍼맥스 파슬실버 낫띵리튼 텔파 아모멘토 시엔느 단톤,"['▁나이키', '▁베이퍼맥스', '▁파슬', '실버', '▁낫띵리튼', '▁텔',...","[41, 2612, 7068, 1267, 13863, 11971, 19004, 14...","['▁나이키', '▁베이퍼맥스', '▁파슬', '실버', '▁낫띵리튼', '▁', ...","[10, 1497, 5105, 685, 11541, 5, 3666, 899, 123..."
...,...,...,...,...,...,...,...,...,...,...
995,3121065,295594.0,♥19900특가♥,NaN,400050999.0,19900특가,"['▁19', '900', '특가']","[513, 3375, 311]","['▁19', '900', '특가']","[223, 3417, 1036]"
996,143235664,75092091.0,롯데백화점 구입 코트,55~66,310110010.0,롯데백화점 구입 코트,"['▁롯데백화점', '▁구입', '▁코트']","[12126, 4237, 127]","['▁롯데백화점', '▁구입', '▁코트']","[9884, 3575, 30]"
997,148465247,3555188.0,여성 자켓,여성자켓,310100999.0,여성 자켓,"['▁여성', '▁자켓']","[31, 52]","['▁여성', '▁자켓']","[13, 11]"
998,150556964,10076676.0,발루찌 남자 블레이저 블레이져 자켓,"남자자켓,블레이져,블레이저,자켓,정장",320090100.0,발루찌 남자 블레이저 블레이져 자켓,"['▁발', '루', '찌', '▁남자', '▁블레이저', '▁블레이져', '▁자켓']","[132, 18956, 18992, 197, 1468, 1103, 52]","['▁발', '루', '찌', '▁남자', '▁블레이저', '▁블레이져', '▁자켓']","[1287, 995, 2881, 55, 732, 506, 11]"


In [21]:
brand = pd.read_csv('./brand extraction dictionary - final.csv')
brand = brand.iloc[:, :3]
brand.columns = ['name', 'eng', 'syn']

In [22]:
brand

,name,eng,syn
0,나이키,nike,조던
1,스톤아일랜드,stone island,NaN
2,뉴발란스,new balance,뉴발
3,아디다스,adidas,NaN
4,노스페이스,north face,"노페, 더 노스페이스"
...,...,...,...
326,타이틀리스트,titleist,NaN
327,오클리,oakley,NaN
328,먼싱웨어,munsingwear,NaN
329,몬츄라,montura,NaN


In [50]:
brand['set'] = brand['name'].fillna('') + ',' + brand['eng'].fillna('') + ',' + brand['syn'].fillna('')

In [51]:
brands = dict(zip(brand['name'], brand['set'].apply(lambda x: str(x).split(','))))

In [52]:
brands

{'MCM': ['MCM', '', '엠씨엠'],
 nan: ['', '', ''],
 '겐조': ['겐조', 'kenzo', '갠조'],
 '고야드': ['고야드', 'goyard', ''],
 '골든구스': ['골든구스', 'golden goose', ''],
 '골든듀': ['골든듀', 'goldendew', ''],
 '구찌': ['구찌', 'gucci', ''],
 '구호': ['구호', 'kuho', ''],
 '그라미치': ['그라미치', 'gramicci', ''],
 '그랜드세이코': ['그랜드세이코', 'grand seiko', ''],
 '그레고리': ['그레고리', 'gregory', ''],
 '까르띠에': ['까르띠에', 'cartier', ''],
 '까스텔바작': ['까스텔바작', 'castel bajac', '까스탤바작'],
 '꼼데가르송': ['꼼데가르송', 'comme des garcons', '꼼데', ' 꼼대', ' 꼼대가르송'],
 '끌로에': ['끌로에', 'chloe', ''],
 '나나미카': ['나나미카', 'nanamica', ''],
 '나이젤카본': ['나이젤카본', 'nigel cabourn', ''],
 '나이키': ['나이키', 'nike', '조던'],
 '낫띵리튼': ['낫띵리튼', 'nothing written', ''],
 '내셔널지오그래픽': ['내셔널지오그래픽',
  'national geographic',
  '네셔널지오그래픽',
  ' 내셔널지오그래피',
  ' 네셔널지오그래피'],
 '널디': ['널디', 'nerdy', ''],
 '넘버나인': ['넘버나인', 'number (n)ine', ''],
 '네이더스': ['네이더스', 'neithers', ''],
 '노스페이스': ['노스페이스', 'north face', '노페', ' 더 노스페이스'],
 '뉴발란스': ['뉴발란스', 'new balance', '뉴발'],
 '뉴에라': ['뉴에라', 'new era', ''],
 '느

In [55]:
brands_token = {}
for k, v in brands.items():
    tmp = [unigram_20000.encode_as_pieces(x) for x in v]
    brands_token[k] = [i for i in tmp if i != []]

In [56]:
brands_token

{'MCM': [['▁', 'MCM'], ['▁엠씨엠']],
 nan: [],
 '겐조': [['▁겐조'], ['▁kenzo'], ['▁갠조']],
 '고야드': [['▁고야드'], ['▁goyard']],
 '골든구스': [['▁골든구스'], ['▁golden', '▁goose']],
 '골든듀': [['▁골든듀'], ['▁golden', 'de', 'w']],
 '구찌': [['▁구찌'], ['▁gucci']],
 '구호': [['▁구호'], ['▁kuho']],
 '그라미치': [['▁그라미치'], ['▁gr', 'ami', 'cci']],
 '그랜드세이코': [['▁그랜드', '세이', '코'], ['▁gr', 'and', '▁seiko']],
 '그레고리': [['▁그레고리'], ['▁gr', 'eg', 'ory']],
 '까르띠에': [['▁까르띠에'], ['▁cartier']],
 '까스텔바작': [['▁까스텔바작'],
  ['▁castel', '▁ba', 'j', 'ac'],
  ['▁까', '스', '탤', '바', '작']],
 '꼼데가르송': [['▁꼼데가르송'],
  ['▁comme', '▁des', '▁garcons'],
  ['▁꼼데'],
  ['▁꼼', '대'],
  ['▁꼼대가르송']],
 '끌로에': [['▁끌로에'], ['▁chloe']],
 '나나미카': [['▁나나미카'], ['▁nan', 'am', 'ica']],
 '나이젤카본': [['▁나이젤카본'], ['▁n', 'ige', 'l', '▁ca', 'b', 'our', 'n']],
 '나이키': [['▁나이키'], ['▁nike'], ['▁조던']],
 '낫띵리튼': [['▁낫띵리튼'], ['▁no', 'thing', '▁w', 'ri', 't', 'ten']],
 '내셔널지오그래픽': [['▁내셔널지오그래픽'],
  ['▁n', 'ation', 'al', '▁ge', 'o', 'graph', 'ic'],
  ['▁네셔널지오그래픽'],
  ['▁내셔널', '지오', '그

In [57]:
len(brands_token)

331

In [87]:
# unigram_20000_pieces = df['preprocessed'].apply(lambda x: unigram_20000.encode_as_pieces(x))
def extract(brand):
    result = []
    bl = brands_token[brand]
    print(bl)
    for i in unigram_20000_pieces:
        for b in bl:
            if all(bi in i for bi in b):
                result.append(i)
                break
    return result

In [88]:
cartier = extract('까르띠에')

[['▁까르띠에'], ['▁cartier']]


In [89]:
len(cartier)

9849

In [85]:
cartier

[['▁까르띠에', '▁드롭', '▁귀걸이'],
 ['▁까르띠에', '▁클', '런치', '백'],
 ['▁까르띠에', '▁산토스', '▁콤비', '▁특', '급', '▁커플가능'],
 ['▁v', '6', '▁까르띠에', '▁발롱블루', '33', 'mm', '▁메탈', '여성시계'],
 ['▁까르띠에', '탱', '크'],
 ['▁까르띠에', '▁남성시계'],
 ['▁까르띠에', '▁풀박스', '▁탱크'],
 ['▁까', '▁', '띠에', '▁드라이브', '▁드', '▁까르띠에', '▁100', '실사'],
 ['▁까르띠에', 'st', '팔찌'],
 ['▁까르띠에', '가죽'],
 ['▁까르띠에', '▁솔로탱크', '▁커플가능', '▁인기'],
 ['▁까르띠에', '▁드라이브', '▁드'],
 ['▁14', 'k', '▁18', 'k', '▁까르띠에', '▁러브팔찌', '▁6', 'mm'],
 ['▁까르띠에', '▁링', '▁귀걸이', '▁골드', '▁실버'],
 ['▁까르띠에', '▁링'],
 ['▁까르띠에', '▁두줄팔찌'],
 ['▁까르띠에', '▁팔찌', '▁골드', '▁로즈골드'],
 ['▁까르띠에', '▁신상', '▁4', '핀', '▁오토', '시계'],
 ['▁까르띠에',
  '▁18',
  'k',
  '▁저스트앵끌루',
  '▁못팔찌',
  '▁신형',
  '▁핑크골드',
  '▁헤드',
  'd',
  'ia',
  '▁퍼펙트',
  '▁제작'],
 ['▁까르띠에',
  '▁전디자인',
  '▁18',
  'k',
  '▁반지',
  '▁목걸이',
  '▁팔찌',
  '▁핑크골드',
  '▁파',
  '베',
  '▁올',
  '다이아',
  '▁퍼펙트'],
 ['▁무료배송', '▁총알배송', '▁까르띠에', '▁쿼츠', '신형', '33', 'mm'],
 ['▁까르띠에', '▁산토스', '▁갈', '베', '실버'],
 ['▁까르띠에', '▁발롱블루', '▁여성용'],
 ['▁까르띠에', '▁선글라스'],
 ['▁까르띠에', '▁팔

In [90]:
commedesgarcons = extract('꼼데가르송')

[['▁꼼데가르송'], ['▁comme', '▁des', '▁garcons'], ['▁꼼데'], ['▁꼼', '대'], ['▁꼼대가르송']]


In [92]:
len(commedesgarcons)

22638

In [93]:
commedesgarcons

[['▁꼼데가르송', '▁가디건'],
 ['▁꼼데가르송', '▁셔츠'],
 ['▁꼼데가르송', '▁반팔', '▁최저가', '▁보장', '▁꼼데', '반팔티'],
 ['▁꼼데', '▁셔츠'],
 ['▁꼼데가르송', '▁여성용', '▁정품', '반팔티'],
 ['▁꼼데가르송', '▁검정색', '▁가디건', '▁재입고', '▁az', '▁n', '008', '▁051', '▁1', '▁4'],
 ['▁꼼데가르송', '▁스트라이프', '▁셔츠', '▁4', '사이즈'],
 ['▁꼼데가르송', '▁굿', '디자인', '샵', '▁블루종', '▁cdg', '▁야구잠바'],
 ['▁꼼데', '▁하트', '▁와팬', '▁백팩'],
 ['▁꼼데가르송', '▁셔츠', '▁s'],
 ['▁꼼데가르송', '가디건', '▁블랙와펜', 'm'],
 ['▁꼼데가르송', '▁가디간'],
 ['▁스톤', '▁꼼데', '▁오프화이트', '▁마르셀로', '▁10', '이하', '▁꿀매', '봐요'],
 ['▁꼼데', '▁반팔'],
 ['▁정품', '▁꼼데가르송', '▁올', '프린팅', '▁블랙', '▁카모', '▁반팔', '▁슬리브', '▁티셔츠'],
 ['▁꼼데가르송', '▁긴팔티'],
 ['▁남녀공용', '▁꼼데', '▁르', '송', '▁st', '▁라운드'],
 ['▁꼼데가르송', '▁긴팔', '보', '더', '티'],
 ['▁꼼데가르송', '▁셔츠', '▁여성용'],
 ['▁꼼데가르송', '▁스트라이프', '▁셔츠'],
 ['▁꼼데가르송', '레플리카'],
 ['▁꼼데가르송', '▁옴므', '플러스', '▁울', '▁코트자켓', '▁m'],
 ['▁새상품', '▁꼼데가르송', '▁도트', '▁긴팔티', '▁반팔티', '▁집업', '▁후드집업', '▁바람막이', '▁맨투맨'],
 ['▁꼼데가르송', '▁빅아이', '▁반팔티셔츠'],
 ['▁꼼데', '▁하트', '▁긴팔', '▁티셔츠'],
 ['▁꼼데', '와이셔츠', '▁꼼데', '흰', '셔츠', '▁쿨거래', '▁쇼핑백', '도

In [94]:
kenzo = extract('겐조')

[['▁겐조'], ['▁kenzo'], ['▁갠조']]


In [95]:
len(kenzo)

11338

In [96]:
kenzo

[['▁겐조', '▁맨투맨'],
 ['▁겐조', '정품'],
 ['▁겐조', '▁플라워', '▁가디건', '▁66'],
 ['▁겐조', '▁실버', '▁자수', '▁타이거'],
 ['▁겐조', '▁숄더백', '▁', '가져가세여'],
 ['▁투명', '▁겐조', '▁클러치백'],
 ['▁겐조', '▁맨투맨'],
 ['▁무료배송', '▁kenzo', '▁타이거', '▁자수', '▁티셔츠'],
 ['▁겐조', '▁파', '정', '▁맨투맨', '▁팝니다'],
 ['▁겐조', '▁퍼퓸', '▁클러치', '▁파우치백'],
 ['▁겐조', '▁자수', '타이거', '▁블랙', '티'],
 ['▁겐조', '▁가디건'],
 ['▁m', '▁겐조', '▁눈알', '▁반팔티'],
 ['▁인기', '▁겐조', 'st', '▁스냅백'],
 ['▁겐조', '▁에스파듀', '▁슬립온'],
 ['▁겐조', '18', 'ss', '맨투맨'],
 ['▁kenzo', '▁반팔티'],
 ['▁겐조', '▁넥타이', '▁정품', '▁택포', '▁18', '▁000'],
 ['▁겐조', '▁반팔티셔츠'],
 ['▁갠조', '▁골드', '자', '주', '▁타이거'],
 ['▁겐조', '클러치', '▁한정판', '▁싸게', '▁드려요', '▁얼마', '▁안', '썻', '어요'],
 ['▁s', '▁겐조', '▁로고', '▁화이트', '▁반팔', '▁판매완료'],
 ['▁겐조', '▁에펠탑', '▁맨투맨', '▁실착', '▁3', '회', '▁미만'],
 ['▁겐조', '▁눈', '▁자수', '▁실켓', '▁반팔', '▁티셔츠'],
 ['▁19', 'ss', '▁겐조', '▁반팔티', '▁매장판', '▁공용'],
 ['▁겐조', '▁타이거', '로', '어', '▁맨투맨'],
 ['▁겐조', '▁맨투맨'],
 ['▁겐조', '▁pk', '티셔츠'],
 ['▁겐조', '▁뉴', '▁타이거', '▁후드'],
 ['▁겐조', 'st', '▁스냅백', '▁주말', '세일'],
 ['▁겐조', '▁클러치'

In [98]:
brand_product_dict = {}
for b in tqdm(brands_token):
    brand_product_dict[b] = extract(b)

  0%|          | 0/331 [00:00<?, ?it/s]

[['▁나이키'], ['▁nike'], ['▁조던']]


  0%|          | 1/331 [00:10<58:16, 10.60s/it]

[['▁스톤아일랜드'], ['▁stone', '▁island']]


  1%|          | 2/331 [00:17<52:41,  9.61s/it]

...

 99%|█████████▉| 329/331 [1:56:47<00:32, 16.05s/it]

[['▁몬츄라'], ['▁mont', 'ura']]


100%|█████████▉| 330/331 [1:56:55<00:13, 13.61s/it]

[['▁마크앤로나'], ['▁mark', '&', 'lon', 'a']]


100%|██████████| 331/331 [1:57:03<00:00, 21.22s/it]


In [99]:
import json 
 
with open("brand_product_dict.json", "w") as outfile: 
    json.dump(brand_product_dict, outfile)

In [100]:
len(brand_product_dict)

331

In [104]:
result_brand_list = []
for k, v in brand_product_dict.items():
    size = len(v)
    if size > 0:
        result_brand_list.append((k, size))
    else:
        print(k)

nan


In [105]:
len(result_brand_list)

330

In [106]:
result_brand_list

[('나이키', 255114),
 ('스톤아일랜드', 29689),
 ('뉴발란스', 42895),
 ('아디다스', 155233),
 ('노스페이스', 37268),
 ('스투시', 12264),
 ('아식스', 6634),
 ('무신사스탠다드', 974),
 ('디스이즈네버댓', 10843),
 ('커버낫', 8172),
 ('메종키츠네', 3909),
 ('아미', 4396),
 ('아더에러', 2023),
 ('반스', 18653),
 ('아크네', 7204),
 ('컨버스', 22875),
 ('유니클로', 39468),
 ('알파인더스트리', 1235),
 ('에센셜', 3643),
 ('내셔널지오그래픽', 5538),
 ('앤더슨벨', 3881),
 ('파타고니아', 5614),
 ('르꼬끄', 19510),
 ('칼하트', 24025),
 ('구찌', 122452),
 ('프라다', 48702),
 ('자라', 44827),
 ('코스', 9347),
 ('메종마르지엘라', 1477),
 ('라퍼지스토어', 1332),
 ('비바스튜디오', 4849),
 ('프라이탁', 2214),
 ('커먼프로젝트', 520),
 ('라코스테', 41167),
 ('챔피온', 17314),
 ('모드나인', 642),
 ('슈프림', 18116),
 ('오베이', 3201),
 ('피어오브갓', 3591),
 ('루이비통', 95571),
 ('페이탈리즘', 854),
 ('뉴에라', 5605),
 ('바버', 2891),
 ('피스워커', 1046),
 ('오프화이트', 19627),
 ('톰브라운', 30268),
 ('보테가베네타', 15783),
 ('디올', 28562),
 ('알렉산더맥퀸', 3573),
 ('무스너클', 11810),
 ('티파니앤코', 1186),
 ('에르메스', 19995),
 ('샤넬', 69650),
 ('토템', 192),
 ('자크뮈스', 142),
 ('단톤', 627),
 ('판도라', 3665),
 ('스푸닝', 

In [108]:
brand_product_dict['롯데자이언츠']

[['▁롯데자이언츠', '▁홍', '성', '흔', '▁유니폼'],
 ['▁xl', '▁아디다스', '▁롯데자이언츠', '▁춘추', '▁야구점퍼', '▁블랙', '75'],
 ['▁롯데자이언츠', '▁챔피언', '▁홈', '▁완전', '▁새상품'],
 ['▁롯데자이언츠', '▁모자', '▁등', '▁판매합니다'],
 ['▁롯데자이언츠', '▁유니폼', '▁아이다스', '▁새제품', '▁울산'],
 ['▁롯데자이언츠', '▁유니폼', '▁', '응', '원', '복', '▁원', '정', '▁팝니다'],
 ['▁실측', '95', '▁아디다스', '▁롯데자이언츠', '▁홍', '성', '흔', '▁', '마킹', '▁유니폼', '▁무료배송'],
 ['▁95', '▁네', '포스', '▁롯데자이언츠', '▁유니폼', '▁무료배송'],
 ['▁100', '▁롯데자이언츠', '▁싸인', '▁유니폼', '▁무료배송'],
 ['▁롯데자이언츠', '▁사', '인', '유니폼'],
 ['▁롯데자이언츠', '▁외투'],
 ['▁롯데자이언츠', '▁모자', '▁팝니다'],
 ['▁롯데자이언츠', '▁유니폼', '▁팔아요'],
 ['▁롯데자이언츠', '▁31', '번', '▁손', '아', '섭', '▁85', '▁유니폼'],
 ['▁롯데자이언츠', '▁김', '민', '혁', '▁유니폼', '▁저지', '▁야구', '복', '▁95'],
 ['▁k', 'bo', '▁뉴에라', '▁59', 'fifty', '▁롯데자이언츠'],
 ['▁롯데자이언츠', '잠바'],
 ['▁롯데자이언츠', '▁47', '▁레플리카', '▁l'],
 ['▁xxl', '▁네', '포스', '▁롯데자이언츠', '▁이', '대', '호', '▁원', '정', '▁유니폼', '184'],
 ['▁여성', '95', '▁롯데자이언츠', '▁이', '대', '호', '▁', '마킹', '▁유니폼', '▁무료배송'],
 ['▁롯데자이언츠', '▁바람막이', '▁90', '▁파격할인'],
 ['▁롯데자이언츠', '▁